In [1]:
dat = [i.strip().split("|") for i in open ("foursquare/checkins.dat").readlines()]
del dat[1]

In [2]:
import csv
with open("checkins.csv", 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(dat)

In [3]:
import pandas as pd

input_data = pd.read_csv("checkins.csv")

/Users/borispetrusev/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
input_data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [5]:
input_data.columns = ['id','user_id','venue_id','latitude','longitude','created_at']

In [6]:
input_data ['latitude'] = input_data['latitude'].str.strip()

In [7]:
input_data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [8]:
input_data = input_data[input_data['latitude']!=""]

In [9]:
input_data = input_data.reset_index()

In [10]:
input_data = input_data[:-2]

In [11]:
len(input_data)

396634

In [12]:
input_data['longitude'] = input_data['longitude'].str.strip()

In [13]:
input_data.dtypes

index           int64
id             object
user_id       float64
venue_id      float64
latitude       object
longitude      object
created_at     object
dtype: object

In [14]:
input_data[['latitude','longitude']] = input_data[['latitude','longitude']].apply(pd.to_numeric)

In [15]:
input_data.describe()

,index,user_id,venue_id,latitude,longitude
count,3.966340e+05,3.966340e+05,3.966340e+05,396634.000000,396634.000000
mean,4.964017e+05,5.641349e+05,1.327559e+05,37.097070,-86.354065
std,3.013401e+05,4.865694e+05,2.287003e+05,8.778760,37.983677
min,1.000000e+00,1.000000e+00,1.000000e+00,-75.250973,-159.670833
25%,2.315962e+05,1.692900e+05,7.620000e+03,33.800745,-111.926052
50%,4.941385e+05,4.298200e+05,2.830400e+04,39.099275,-85.758456
75%,7.578510e+05,8.496470e+05,1.485520e+05,40.802071,-74.056530
max,1.021964e+06,2.153361e+06,1.143011e+06,78.218590,178.424240


In [16]:
X = input_data[['latitude','longitude']][:100000]
print(X.head())
print(X.shape)

    latitude   longitude
0  38.895112  -77.036366
1  33.800745  -84.410520
2  45.523452 -122.676207
3  40.764462 -111.904565
4  33.448377 -112.074037
(100000, 2)


In [17]:
from sklearn.cluster import MeanShift
import numpy as np

In [18]:
bandwidth = 0.1
ms = MeanShift(bandwidth=bandwidth)
ms.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [19]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters = len(labels_unique)
print(n_clusters)

3230


In [22]:
X['cluster'] = labels

In [23]:
X['cluster_count'] = X.groupby('cluster')['cluster'].transform('count')

In [24]:
X_target = X[X['cluster_count']>15]

In [25]:
X_target.shape
X_target = X_target.reset_index()

In [26]:
offices = pd.DataFrame([[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],
                       [52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]])

In [27]:
offices.columns=[['latitude','longitude']]
print(offices)

    latitude   longitude
0  33.751277 -118.188740
1  25.867736  -80.324116
2  51.503016   -0.075479
3  52.378894    4.885084
4  39.366487  117.036146
5 -33.868457  151.205134


In [ ]:
large_clusters = X_target['cluster'].unique()
print(len(large_clusters))
cluster_centers = pd.DataFrame(cluster_centers)
cluster_centers.columns = [['latitude','longitude']]
cluster_centers['filter']=""

In [41]:
for i in range(0, len(cluster_centers)):
    if i in large_clusters:
        cluster_centers['filter'][i]=1
print(cluster_centers)

/Users/borispetrusev/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


       latitude   longitude filter
0     40.717716  -73.991835      1
1     33.449438 -112.002140      1
2     33.446380 -111.901888      1
3     41.878244  -87.629843      1
4     37.688682 -122.409330      1
5     38.886165  -77.048783      1
6     33.357345 -111.822654      1
7     33.766636  -84.393289      1
8     42.363219  -71.073688      1
9     47.606245 -122.332044      1
10    36.117229 -115.171073      1
11    34.060398 -118.248709      1
12    44.977948  -93.267301      1
13    30.267184  -97.743119      1
14    40.766876  -73.833353      1
15    39.735830 -104.986580      1
16    39.951680  -75.162736      1
17    34.035487 -118.438998      1
18    32.980893 -117.078118      1
19    32.803021  -96.769897      1
20    37.347871 -121.947287      1
21    28.543502  -81.376429      1
22    32.711344 -117.153639      1
23    32.221713 -110.926535      1
24    34.127402 -118.351884      1
25    29.762698  -95.382314      1
26    43.040533  -87.914333      1
27    33.817306 -117

In [44]:
interesting_clusters = cluster_centers[cluster_centers['filter']==1]
print(interesting_clusters.shape)

(593, 3)


In [49]:
for i in range(0, len(offices)):
    interesting_clusters['dist_'+str(i)]=""
interesting_clusters = interesting_clusters.reset_index()
print(interesting_clusters)

     index   latitude   longitude filter dist_0 dist_1 dist_2 dist_3 dist_4  \
0        0  40.717716  -73.991835      1                                      
1        1  33.449438 -112.002140      1                                      
2        2  33.446380 -111.901888      1                                      
3        3  41.878244  -87.629843      1                                      
4        4  37.688682 -122.409330      1                                      
5        5  38.886165  -77.048783      1                                      
6        6  33.357345 -111.822654      1                                      
7        7  33.766636  -84.393289      1                                      
8        8  42.363219  -71.073688      1                                      
9        9  47.606245 -122.332044      1                                      
10      10  36.117229 -115.171073      1                                      
11      11  34.060398 -118.248709      1            

/Users/borispetrusev/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
import geopy.distance
for i in range(0, len(interesting_clusters)):
    for z in range(0, len(offices)):
        coord_1 = (X_target['latitude'][i], X_target['longitude'][i])
        coord_2 = (offices['latitude'][z], offices['longitude'][z])
        interesting_clusters['dist_'+str(z)][i] = geopy.distance.vincenty(coord_1, coord_2).km

/Users/borispetrusev/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
min_dist = []
for i in range(0, len(offices)):
    min_dist.append(min(interesting_clusters['dist_'+str(i)]))
print(min_dist)
min_dist_v = min(min_dist)
interesting_clusters.loc[interesting_clusters['dist_5'] == min_dist_v]

[6.518653168689608, 12.529974454236658, 3.538854552963338, 355.4910213749812, 889.8505917135872, 3.312987248260675]


,index,latitude,longitude,filter,dist_0,dist_1,dist_2,dist_3,dist_4,dist_5
410,413,-33.86063,151.204776,1,12050.1,15021.4,16983.7,16635.7,8836.78,3.31299
